In [35]:
import fitz  # PyMuPDF
import easyocr
from tenacity import retry, stop_after_attempt, wait_random_exponential
import requests
import json
import os

In [36]:
api_key = os.getenv("MISTRAL_API_KEY")

In [29]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def clean_text(extraction):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    payload = {
        "model": "mistral-small-latest",
        "messages": [
            {
                "role": "user",
                "content": """
                    Du bist ein Experte für Textsäuberung von Bankauszügen. Bitte gebe die folgenden Inhalte als Markdown wieder, sodass ich dieses markdown direkt kopieren und verwenden kann. Bitte benutze keine Tabelle, sonders Liste alle zusammengehörenden Elemente als Block auf. Ein Block sollte so aussehen: 
                    ### Buchung X:
                    - Auftraggeber/Empfänger:
                    - Saldo: 
                    - Betrag:
                    - Valuta: 
                    - Buchungstext:
                    - Verwendungszweck: 

                    Alle weiteren Informationen, welche nicht Buchungen sind, sollen auch als Block dargestellt werden. 
                """.strip()
            },
            {
                "role": "user",
                "content": f"<<< Säubere diesen Text: {extraction} >>>"
            }
        ],
        "max_tokens": 4096,	
        "temperature": 0.0
    }

    response = requests.post("https://api.mistral.ai/v1/chat/completions", headers=headers, data=json.dumps(payload))
    response.raise_for_status()  # Raise an exception for HTTP errors
    response_object = response.json()
    print(response_object)
    answer = response_object["choices"][0]["message"]["content"]
    return answer

In [30]:
def extract_text_from_pdf(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)

    # Initialize EasyOCR reader
    reader = easyocr.Reader(['de'])  # Change 'en' to the language you want to extract

    # Iterate over each page
    for page_index in range(3):
        # Get the page
        page = doc[page_index]
        

        # Convert the page to an image
        pix = page.get_pixmap(dpi=300)
        pix.save("temp_page.png")  # Save as a temporary image

        # Extract text from the image
        result = reader.readtext("temp_page.png")

        # Print the extracted text
        page_content = []
        for text_block in result:
            page_content.append(text_block[1])
        page_text = "\n".join(page_content)
        print(f"Page {page_index + 1}:\n{page_text}")
        print("\n -------- \n")
        cleaned_text = clean_text(page_text)
        print(f"Cleaned Page {page_index + 1}:\n{cleaned_text}")
        print("\n -------- \n")

In [31]:
# Call the function with the path to your PDF
extract_text_from_pdf("Umsatzanzeige_DE06500105175423175609_20240615.pdf")

Page 1:
ING
1
Umsatzanzeige
9
;
g
Leonard Püttmann
Erstellt am
15.06.2024, 20:51 Uhr
Meertal 32
Letztes Konto Update
1
41464 Neuss
Bank
ING
Kontoname
Girokonto
{
IBAN
DEO6 5001 0517 5423 1756 09
p
Eingeräumte Kontoüberziehung
0,00
9
Saldo
516,76 €
8
;
Auswahlkriterien
;

Zeitraum
Filter
Sortierung
Transaktionen
15.06.2023 bis 15.06.2024
Datum absteigend
1191
Im PDF finden Sie alle bereits gebuchten Umsätze: Die vorgemerkten Umsätze werden nicht aufgenommen; auch
wenn sie in Ihrem Internetbanking angezeigt werden:
Buchung
Auftraggeber/Empfänger
Saldo
Betrag
Valuta
Buchungstext
Verwendungszweck
Notiz
17.06.2024
Leonard Puettmann
516,76 €
-0,57 €
17.06.2024
Überweisung
14.06.2024
VISA EDEKA PRELLER 3441
517,33 €
-6,57 €
14.06.2024
Lastschrift
NR XXXX 9017 BRECKERFELD DEKAUFUMSATZ12.06
6.57195707ARN743960441-
64060005577235Google
14.06.2024
VISA Q PARK EUROPARKING
523,90 €
-100,00 €
14.06.2024
Lastschrift
NR XXXX 9017 AMSTERDAM NLKAUFUMSATZI2.06
100.00090717ARN74662794165449008396766Google

In [ ]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('my_database.db')

# Create a cursor object
c = conn.cursor()

# Create a table
c.execute('''
    CREATE TABLE IF NOT EXISTS my_table
    (Auftraggeber_Empfänger TEXT, Saldo REAL, Betrag REAL, Valuta TEXT, Buchungstext TEXT, Verwendungszweck TEXT)
''')

# Insert data into the table
data = [
    ('John Doe', 1000.0, 200.0, 'USD', 'Some text', 'Some purpose'),
    # Add more data tuples here
]

for item in data:
    c.execute('''
        INSERT INTO my_table (Auftraggeber_Empfänger, Saldo, Betrag, Valuta, Buchungstext, Verwendungszweck)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', item)

# Commit the changes and close the connection
conn.commit()
conn.close()